In [ ]:
!nvidia-smi

Thu Nov  7 05:12:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.1/112.6 GB disk)


In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n-seg.pt')
image = Image.open('5.jpg')
result = model.predict(image, conf=0.25)[0]


0: 384x640 (no detections), 83.6ms
Speed: 21.7ms preprocess, 83.6ms inference, 103.3ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("aditya-choudhary-ehv9p").project("l-s-kvbur")
version = project.version(1)
dataset = version.download("yolov11")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to L-S-1 in yolov11:: 100%|██████████| 2050/2050 [00:00<00:00, 3068.40it/s]


In [ ]:
dataset.location

'/content/datasets/L-S-1'

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolo11m-seg.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True

/content
100% 43.3M/43.3M [00:00<00:00, 363MB/s]
Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=/content/datasets/L-S-1/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

In [ ]:
!yolo segment predict model=yolo11n-seg.pt source='/content/5.jpg'  # predict with official model

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs

image 1/1 /content/5.jpg: 384x640 (no detections), 64.8ms
Speed: 3.9ms preprocess, 64.8ms inference, 54.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo segment predict model=/content/runs/segment/train2/weights/best.pt source='/content/38244.jpg'  # predict with custom model

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m-seg summary (fused): 330 layers, 22,336,854 parameters, 0 gradients, 123.0 GFLOPs

image 1/1 /content/38244.jpg: 384x640 2 lane1s, 1 lane2, 77.3ms
Speed: 3.0ms preprocess, 77.3ms inference, 932.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict5
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo detect predict model=yolo11n.pt source='/content/38235.jpg'

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /content/38235.jpg: 384x640 1 car, 52.2ms
Speed: 3.0ms preprocess, 52.2ms inference, 727.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
